In [1]:
path = "PRIMNET/FINGER"

from path_handler import get_BASERDIR
import os
BASEDIR, RUNMODE = get_BASERDIR(".")

if BASEDIR.absolute().name == "control":
    os.chdir("..")
os.getcwd()

Running on Jupyter...


'/home/taerim/PRIMNET-V2/code'

In [2]:
# %%
from pathlib import Path
import sys

try: 
    get_ipython().__class__.__name__
    BASEDIR = Path().absolute()
except: BASEDIR = Path(__file__).parent

sys.path.append(str(BASEDIR))
import torch
from torch import nn
import argparse

import numpy as np
from utils import dataloader
from utils.initalize import INITALZE_EVEN_JOINTS
from utils.update import  update_primnet, update_fc_primnet, update_pcc_primnet


import os
import random
from pathlib import Path
import wandb
import time
import json

from utils.tools import set_seed, set_wandb, print_log_dict, prefix_dict, average_dict
from utils.path_handler import JUPYTER, RUN, DEBUG, get_BASERDIR
from utils.args import read_ARGS
from utils.logger import CSVLogger,ask_and_make_folder
from utils.tools import cast_numpy

from configs.template import PRIMNET_ARGS_TEMPLATE, FC_PRIMNET_ARGS_TEMPLATE, PCC_PRIMNET_ARGS_TEMPLATE
from model.PRIMNET import PRIMNET
from model.FC_PRIMNET import FC_PRIMNET
from typing import Union

from utils.dataloader import get_dataset, Sampler

In [3]:
BASEDIR, RUNMODE = get_BASERDIR(os.getcwd())
args = read_ARGS((BASEDIR/'results'/path/"args.py").absolute())
args

Running on Jupyter...


ARGS(MODEL='PRIMNET', EVEN_JOINTS=True, WANDB=True, pname='PRIMNET_v2.1', runname='FINGER', DATASET='FINGER', TPOSE=((0, 0, 0.12),), LOAD_WEIGHTPATH=None, SAVE_PERIOD=1, TEST_PERIOD=1, EVEN_JOINT=True, p_offset_std=0.1, rpy_offset_std=0.01, axis_std=0.1, OUTPUT_NORMALIZE=False, seed=0, hdim=(16, 16), motor_embed_dim=4, lr=0.0015, lrd=0.95, wd=0.0, w_vec=0.001, epochs=500, focus_ratio=0.0, data_ratio=1.0, n_workers=2, batch_size=64, joint_seqs=('F', 'R', 'P', 'R', 'R', 'P', 'R'), marker_num=1, motor_dim=4)

In [4]:
model = PRIMNET(args=args).to(args.device)

model.load_state_dict(torch.load(BASEDIR/'results'/path/"weights/epoch_500.pth"))

<All keys matched successfully>

In [5]:
def forward_motor(model:PRIMNET, motor_control):     
    motor_control = model.normalize(motor_control)
    
    # Forward
    act_embeds = model.ACT_EMBED.layers(motor_control)
    q_values = model.FK_LAYER.forward_q(act_embeds)
    joint_se3 = model.FK_LAYER.forward_kinematics(q_values)
    
    return model.t2p(joint_se3, OUTPUT_NORMALIZE=False)



## Kinematic gradient - Autograd

In [6]:
from torch.autograd.functional import _vmap,fwAD, _as_tuple, _grad_preprocess, _check_requires_grad, _construct_standard_basis_for, _autograd_grad, _grad_postprocess, _tuple_postprocess

def _jacfwd(func, inputs, strict=False, vectorize=False):
    if strict:
        raise RuntimeError('torch.autograd.functional.jacobian: `strict=True` '
                           'and `strategy="forward-mode"` are not supported together (yet). '
                           'Please either set `strict=False` or '
                           '`strategy="reverse-mode"`.')
    is_inputs_tuple, inputs = _as_tuple(inputs, "inputs", "jacobian")
    output_info = []

    if vectorize:
        # See NOTE: [Computing jacobian with vmap and grad for multiple outputs]
        input_numels = tuple(input.numel() for input in inputs)

        # Step 1: Prepare tangents
        tangents = _construct_standard_basis_for(inputs, input_numels)

        # Step 2: Compute vmap over computation with dual tensors
        def jvp(tangents):
            with fwAD.dual_level():
                dual_inputs = tuple(
                    fwAD.make_dual(input, tangent.view_as(input)) for input, tangent in zip(inputs, tangents))
                _is_outputs_tuple, dual_outputs = _as_tuple(func(*dual_inputs), "outputs")
                output_info.append(_is_outputs_tuple)
                jv = []
                primal_outs = []
                for dual_out in dual_outputs:
                    primal, tangent = fwAD.unpack_dual(dual_out)
                    primal_outs.append(primal)
                    if tangent is not None:
                        jv.append(tangent)
                    else:
                        jv.append(torch.zeros_like(primal))
                output_info.append(primal_outs)
                return tuple(jv)

        outputs_before_split = _vmap(jvp)(tangents)
        is_outputs_tuple, outputs = output_info
        # Step 3: for each of the output tangents, split along dim 0
        jacobian_input_output = []
        for jac, output_i in zip(outputs_before_split, outputs):
            jacobian_output_i_output = []
            for jac, input_j in zip(jac.split(input_numels, dim=0), inputs):
                # We need to transpose the Jacobian because in forward AD, the
                # batch dimension represents that of the inputs
                jacobian_input_i_output_j = jac.permute(*range(1, jac.ndim), 0) \
                    .reshape(tuple([*output_i.shape, *input_j.shape]))  # noqa: C409

                jacobian_output_i_output.append(jacobian_input_i_output_j)
            jacobian_input_output.append(jacobian_output_i_output)

        # Omit [Step 4] because everything is already transposed w/ forward AD
        return _tuple_postprocess(jacobian_input_output, (is_outputs_tuple, is_inputs_tuple))
    else:
        raise NotImplementedError("Computing Jacobian using forward-AD or forward-over-reverse Hessian is"
                                  "only implemented for `vectorize=True`.")

In [7]:
from torch.autograd.functional import _jacfwd, _as_tuple, _grad_preprocess, _check_requires_grad, _construct_standard_basis_for, _autograd_grad, _grad_postprocess, _tuple_postprocess

from typing import List, Tuple
def jacobian(func, inputs, create_graph=False, strict=False, vectorize=False, strategy="reverse-mode"):
    r"""Function that computes the Jacobian of a given function.

    Args:
        func (function): a Python function that takes Tensor inputs and returns
            a tuple of Tensors or a Tensor.
        inputs (tuple of Tensors or Tensor): inputs to the function ``func``.
        create_graph (bool, optional): If ``True``, the Jacobian will be
            computed in a differentiable manner. Note that when ``strict`` is
            ``False``, the result can not require gradients or be disconnected
            from the inputs.  Defaults to ``False``.
        strict (bool, optional): If ``True``, an error will be raised when we
            detect that there exists an input such that all the outputs are
            independent of it. If ``False``, we return a Tensor of zeros as the
            jacobian for said inputs, which is the expected mathematical value.
            Defaults to ``False``.
        vectorize (bool, optional): This feature is experimental.
            Please consider using
            `functorch's jacrev or jacfwd <https://github.com/pytorch/functorch#what-are-the-transforms>`_
            instead if you are looking for something less experimental and more performant.
            When computing the jacobian, usually we invoke
            ``autograd.grad`` once per row of the jacobian. If this flag is
            ``True``, we perform only a single ``autograd.grad`` call with
            ``batched_grad=True`` which uses the vmap prototype feature.
            Though this should lead to performance improvements in many cases,
            because this feature is still experimental, there may be performance
            cliffs. See :func:`torch.autograd.grad`'s ``batched_grad`` parameter for
            more information.
        strategy (str, optional): Set to ``"forward-mode"`` or ``"reverse-mode"`` to
            determine whether the Jacobian will be computed with forward or reverse
            mode AD. Currently, ``"forward-mode"`` requires ``vectorized=True``.
            Defaults to ``"reverse-mode"``. If ``func`` has more outputs than
            inputs, ``"forward-mode"`` tends to be more performant. Otherwise,
            prefer to use ``"reverse-mode"``.

    Returns:
        Jacobian (Tensor or nested tuple of Tensors): if there is a single
        input and output, this will be a single Tensor containing the
        Jacobian for the linearized inputs and output. If one of the two is
        a tuple, then the Jacobian will be a tuple of Tensors. If both of
        them are tuples, then the Jacobian will be a tuple of tuple of
        Tensors where ``Jacobian[i][j]`` will contain the Jacobian of the
        ``i``\th output and ``j``\th input and will have as size the
        concatenation of the sizes of the corresponding output and the
        corresponding input and will have same dtype and device as the
        corresponding input. If strategy is ``forward-mode``, the dtype will be
        that of the output; otherwise, the input.

    Example:

        >>> def exp_reducer(x):
        ...   return x.exp().sum(dim=1)
        >>> inputs = torch.rand(2, 2)
        >>> # xdoctest: +IGNORE_WANT("non-deterministic")
        >>> jacobian(exp_reducer, inputs)
        tensor([[[1.4917, 2.4352],
                 [0.0000, 0.0000]],
                [[0.0000, 0.0000],
                 [2.4369, 2.3799]]])

        >>> jacobian(exp_reducer, inputs, create_graph=True)
        tensor([[[1.4917, 2.4352],
                 [0.0000, 0.0000]],
                [[0.0000, 0.0000],
                 [2.4369, 2.3799]]], grad_fn=<ViewBackward>)

        >>> def exp_adder(x, y):
        ...   return 2 * x.exp() + 3 * y
        >>> inputs = (torch.rand(2), torch.rand(2))
        >>> jacobian(exp_adder, inputs)
        (tensor([[2.8052, 0.0000],
                [0.0000, 3.3963]]),
         tensor([[3., 0.],
                 [0., 3.]]))
    """
    assert strategy in ("forward-mode", "reverse-mode"), (
        'Expected strategy to be either "forward-mode" or "reverse-mode". Hint: If your '
        'function has more outputs than inputs, "forward-mode" tends to be more performant. '
        'Otherwise, prefer to use "reverse-mode".')
    if strategy == "forward-mode":
        if create_graph:
            raise NotImplementedError('torch.autograd.functional.jacobian: `create_graph=True` '
                                      'and `strategy="forward-mode"` are not supported together (yet). '
                                      'Please either set `create_graph=False` or '
                                      '`strategy="reverse-mode"`.')
        return _jacfwd(func, inputs, strict, vectorize)

    with torch.enable_grad():
        is_inputs_tuple, inputs = _as_tuple(inputs, "inputs", "jacobian")
        inputs = _grad_preprocess(inputs, create_graph=create_graph, need_graph=True)

        outputs = func(*inputs)
        is_outputs_tuple, outputs = _as_tuple(outputs,
                                              "outputs of the user-provided function",
                                              "jacobian")
        _check_requires_grad(outputs, "outputs", strict=strict)

        if vectorize:
            if strict:
                raise RuntimeError('torch.autograd.functional.jacobian: `strict=True` '
                                   'and `vectorized=True` are not supported together. '
                                   'Please either set `strict=False` or '
                                   '`vectorize=False`.')
            # NOTE: [Computing jacobian with vmap and grad for multiple outputs]
            #
            # Let's consider f(x) = (x**2, x.sum()) and let x = torch.randn(3).
            # It turns out we can compute the jacobian of this function with a single
            # call to autograd.grad by using vmap over the correct grad_outputs.
            #
            # Firstly, one way to compute the jacobian is to stack x**2 and x.sum()
            # into a 4D vector. E.g., use g(x) = torch.stack([x**2, x.sum()])
            #
            # To get the first row of the jacobian, we call
            # >>> autograd.grad(g(x), x, grad_outputs=torch.tensor([1, 0, 0, 0]))
            # To get the 2nd row of the jacobian, we call
            # >>> autograd.grad(g(x), x, grad_outputs=torch.tensor([0, 1, 0, 0]))
            # and so on.
            #
            # Using vmap, we can vectorize all 4 of these computations into one by
            # passing the standard basis for R^4 as the grad_output.
            # vmap(partial(autograd.grad, g(x), x))(torch.eye(4)).
            #
            # Now, how do we compute the jacobian *without stacking the output*?
            # We can just split the standard basis across the outputs. So to
            # compute the jacobian of f(x), we'd use
            # >>> autograd.grad(f(x), x, grad_outputs=_construct_standard_basis_for(...))
            # The grad_outputs looks like the following:
            # ( torch.tensor([[1, 0, 0],
            #                 [0, 1, 0],
            #                 [0, 0, 1],
            #                 [0, 0, 0]]),
            #   torch.tensor([[0],
            #                 [0],
            #                 [0],
            #                 [1]]) )
            #
            # But we're not done yet!
            # >>> vmap(partial(autograd.grad(f(x), x, grad_outputs=...)))
            # returns a Tensor of shape [4, 3]. We have to remember to split the
            # jacobian of shape [4, 3] into two:
            # - one of shape [3, 3] for the first output
            # - one of shape [   3] for the second output

            # Step 1: Construct grad_outputs by splitting the standard basis
            output_numels = tuple(output.numel() for output in outputs)
            grad_outputs = _construct_standard_basis_for(outputs, output_numels)
            flat_outputs = tuple(output.reshape(-1) for output in outputs)

            # Step 2: Call vmap + autograd.grad
            def vjp(grad_output):
                vj = list(_autograd_grad(flat_outputs, inputs, grad_output, create_graph=create_graph, is_grads_batched=True))
                for el_idx, vj_el in enumerate(vj):
                    if vj_el is not None:
                        continue
                    vj[el_idx] = torch.zeros_like(inputs[el_idx]).expand((sum(output_numels),) + inputs[el_idx].shape)
                return tuple(vj)

            jacobians_of_flat_output = vjp(grad_outputs)

            # Step 3: The returned jacobian is one big tensor per input. In this step,
            # we split each Tensor by output.
            jacobian_input_output = []
            for jac, input_i in zip(jacobians_of_flat_output, inputs):
                jacobian_input_i_output = []
                for jac, output_j in zip(jac.split(output_numels, dim=0), outputs):
                    jacobian_input_i_output_j = jac.view(output_j.shape + input_i.shape)
                    jacobian_input_i_output.append(jacobian_input_i_output_j)
                jacobian_input_output.append(jacobian_input_i_output)

            # Step 4: Right now, `jacobian` is a List[List[Tensor]].
            # The outer List corresponds to the number of inputs,
            # the inner List corresponds to the number of outputs.
            # We need to exchange the order of these and convert to tuples
            # before returning.
            jacobian_output_input = tuple(zip(*jacobian_input_output))

            jacobian_output_input = _grad_postprocess(jacobian_output_input, create_graph)
            return _tuple_postprocess(jacobian_output_input, (is_outputs_tuple, is_inputs_tuple))

        jacobian: Tuple[torch.Tensor, ...] = tuple()

        for i, out in enumerate(outputs):

            # mypy complains that expression and variable have different types due to the empty list
            jac_i: Tuple[List[torch.Tensor]] = tuple([] for _ in range(len(inputs)))  # type: ignore[assignment]
            for j in range(out.nelement()):
                vj = _autograd_grad((out.reshape(-1)[j],), inputs,
                                    retain_graph=True, create_graph=create_graph)

                for el_idx, (jac_i_el, vj_el, inp_el) in enumerate(zip(jac_i, vj, inputs)):
                    if vj_el is not None:
                        if strict and create_graph and not vj_el.requires_grad:
                            msg = ("The jacobian of the user-provided function is "
                                   "independent of input {}. This is not allowed in "
                                   "strict mode when create_graph=True.".format(i))
                            raise RuntimeError(msg)
                        jac_i_el.append(vj_el)
                    else:
                        if strict:
                            msg = ("Output {} of the user-provided function is "
                                   "independent of input {}. This is not allowed in "
                                   "strict mode.".format(i, el_idx))
                            raise RuntimeError(msg)
                        jac_i_el.append(torch.zeros_like(inp_el))

            jacobian += (tuple(torch.stack(jac_i_el, dim=0).view(out.size()  # type: ignore[operator]
                         + inputs[el_idx].size()) for (el_idx, jac_i_el) in enumerate(jac_i)), )

        jacobian = _grad_postprocess(jacobian, create_graph)

        return _tuple_postprocess(jacobian, (is_outputs_tuple, is_inputs_tuple)), outputs

In [8]:
motor_control = torch.ones(1,args.motor_dim).to(args.device)

with torch.no_grad():
    act_embeds = model.ACT_EMBED.layers(motor_control)
    q_values = model.FK_LAYER.forward_q(act_embeds)

q_values = q_values.detach()
q_values.requires_grad_(True)


tensor([[ 0.0000, -0.0052, -0.0211,  0.0104, -0.0003, -0.0076, -0.0046]],
       requires_grad=True)

In [9]:
from torch import autograd, Tensor

def forward_kinematics(q_values:Tensor, idx=-1):
    assert q_values.ndim == 2
    assert q_values.shape[0] == 1
    
    joint_se3 = model.FK_LAYER.forward_kinematics(q_values)
    joint_position = model.t2p(joint_se3, OUTPUT_NORMALIZE=False)

    # aux_joints = len(args.joint_seqs)//args.marker_num

    # marker_position = []
    # for i in range(len(args.joint_seqs)):
    #     if (i+1)%aux_joints == 0:
    #         joint_position_ = joint_position[:,i]
    #         marker_position.append(joint_position_[0])

    # return torch.stack(marker_position, dim=0)[prim_idxs]
    return joint_position[0,idx,:,0]

EE_pos = forward_kinematics(q_values, idx=-1)

EE_pos.shape, q_values.shape

(torch.Size([3]), torch.Size([1, 7]))

In [10]:
from functools import partial


jac_auto, val = jacobian(partial(forward_kinematics, idx=6), q_values)
jac_auto = jac_auto[:,0,:]
jac_auto.shape, jac_auto, val

(torch.Size([3, 7]),
 tensor([[ 0.0000,  1.5545,  0.4118,  2.1619, -0.1977, -1.0993,  0.0000],
         [ 0.0000,  1.0748,  1.3707,  1.0633,  1.6178, -2.5206,  0.0000],
         [ 0.0000,  0.2390,  0.1161,  0.2985,  0.2746,  0.1201,  0.0000]]),
 (tensor([ 0.0045, -0.0215,  0.1012], grad_fn=<SelectBackward0>),))

# Kinematics gradient - Explicit

In [11]:
from utils.pyart import t2r

motor_control = model.normalize(motor_control)

# Forward
act_embeds = model.ACT_EMBED.layers(motor_control)
q_values = model.FK_LAYER.forward_q(act_embeds)
joint_se3 = model.FK_LAYER.forward_kinematics(q_values)

joint_position =  model.t2p(joint_se3, OUTPUT_NORMALIZE=False)[0,:,:,0]
joint_rotation = t2r(joint_se3[0])

EE_pos = joint_position[-1]

EE_pos.shape

torch.Size([3])

In [12]:
from model.PRIMNET import Fjoint, Tjoint, Rjoint, Pjoint

jac_explicit = torch.zeros(3, len(model.FK_LAYER.joints))

for idx,joint in enumerate(model.FK_LAYER.joints):
    if isinstance(joint, Fjoint):
        continue
    elif isinstance(joint, Tjoint):
        continue
    elif isinstance(joint, Rjoint):
        pos_diff = EE_pos - joint_position[idx]
        jac_explicit[:, idx] = torch.cross(joint_rotation[idx] @ joint.axis.data[:,0], pos_diff)
        # print('here')
    elif isinstance(joint, Pjoint):
        pos_diff = EE_pos - joint_position[idx]
        jac_explicit[:,idx] = joint_rotation[idx] @ joint.axis.data[:,0]

assert torch.sum(jac_explicit - jac_auto) < 1e-5

## Time Check - Kinematic gradient

In [13]:
import time
n = 100

start_time = time.time()
for _ in range(n):
    jac_auto = jacobian(partial(forward_kinematics, idx=6), q_values)[0]
    jac_auto = jac_auto[:,0,:]
end_time = time.time()

autograd_time = end_time-start_time
print("[AutoGrad]:{:2f}".format(autograd_time))

start_time = time.time()
for _ in range(n):
    jac_explicit = torch.zeros(3, len(model.FK_LAYER.joints))

    with torch.no_grad():
        act_embeds = model.ACT_EMBED.layers(motor_control)
        q_values = model.FK_LAYER.forward_q(act_embeds)
        joint_se3 = model.FK_LAYER.forward_kinematics(q_values)

        joint_position =  model.t2p(joint_se3, OUTPUT_NORMALIZE=False)[0,:,:,0]

        EE_pos = joint_position[-1]

    for idx,joint in enumerate(model.FK_LAYER.joints):
        if isinstance(joint, Fjoint):
            continue
        elif isinstance(joint, Tjoint):
            continue
        elif isinstance(joint, Rjoint):
            pos_diff = EE_pos - joint_position[idx]
            jac_explicit[:, idx] = torch.cross(joint_rotation[idx] @ joint.axis.data[:,0], pos_diff)
            # print('here')
        elif isinstance(joint, Pjoint):
            pos_diff = EE_pos - joint_position[idx]
            jac_explicit[:,idx] = joint_rotation[idx] @ joint.axis.data[:,0]
end_time = time.time()
explicit_time = end_time-start_time
print("[Explicit]:{:2f}".format(explicit_time))

print("Result: [Explicit] is faster than [AutoGrad] by {:2f} times".format(autograd_time/explicit_time))



[AutoGrad]:0.794041
[Explicit]:0.345913
Result: [Explicit] is faster than [AutoGrad] by 2.295493 times


# Time Check - forward w/wo autograd

In [14]:
n = 100

start_time = time.time()
for _ in range(n):
    forward_kinematics(q_values, idx=6)
end_time = time.time()

with_grad_time = end_time-start_time

start_time = time.time()
for _ in range(n):
    with torch.no_grad():
        forward_kinematics(q_values, idx=6)
end_time = time.time()
without_grad_time = end_time-start_time

print("Result: [without_grad] is faster than [with_grad] by {:2f} times".format(with_grad_time/without_grad_time))

Result: [without_grad] is faster than [with_grad] by 1.368354 times


# Full gradient - Autograd

In [15]:
motor_control = torch.ones(1,args.motor_dim).to(args.device)


def forward_model(model:PRIMNET, motor_control):
    motor_control = model.normalize(motor_control)
    
    # Forward
    act_embeds = model.ACT_EMBED.layers(motor_control)
    q_values = model.FK_LAYER.forward_q(act_embeds)
    joint_se3 = model.FK_LAYER.forward_kinematics(q_values)
    
    return model.t2p(joint_se3, OUTPUT_NORMALIZE=False)[0,-1,:,0]


dp_dm, p = jacobian(partial(forward_model, model), motor_control)
dp_dm = dp_dm[:,0,:]
dp_dm, dp_dm.shape

(tensor([[-2.3906e-05,  1.5433e-05,  2.1243e-05, -3.0690e-05],
         [ 8.0282e-06, -5.1830e-06, -7.1341e-06,  1.0307e-05],
         [ 2.8990e-06, -1.8716e-06, -2.5761e-06,  3.7217e-06]]),
 torch.Size([3, 4]))

# Full gradient - Hybrid

In [16]:
def forward_q(model:PRIMNET, motor_control):
    motor_control = model.normalize(motor_control)

    # Forward
    act_embeds = model.ACT_EMBED.layers(motor_control)
    q_values = model.FK_LAYER.forward_q(act_embeds)[0]
    return q_values

dq_dm, q_values_tuple = jacobian(partial(forward_q, model), motor_control)
dq_dm = dq_dm[:,0,:]; q_values = q_values_tuple[0].unsqueeze(0)
dq_dm.shape, q_values.shape

(torch.Size([7, 4]), torch.Size([1, 7]))

In [17]:
from utils.pyart import t2r
joint_se3 = model.FK_LAYER.forward_kinematics(q_values)

joint_position =  model.t2p(joint_se3, OUTPUT_NORMALIZE=False)[0,:,:,0]
joint_rotation = t2r(joint_se3[0])

EE_pos = joint_position[-1]

EE_pos.shape

torch.Size([3])

In [18]:
from model.PRIMNET import Fjoint, Tjoint, Rjoint, Pjoint

jac_explicit = torch.zeros(3, len(model.FK_LAYER.joints))

for idx,joint in enumerate(model.FK_LAYER.joints):
    if isinstance(joint, Fjoint):
        continue
    elif isinstance(joint, Tjoint):
        continue
    elif isinstance(joint, Rjoint):
        pos_diff = EE_pos - joint_position[idx]
        jac_explicit[:, idx] = torch.cross(joint_rotation[idx] @ joint.axis.data[:,0], pos_diff)
        # print('here')
    elif isinstance(joint, Pjoint):
        pos_diff = EE_pos - joint_position[idx]
        jac_explicit[:,idx] = joint_rotation[idx] @ joint.axis.data[:,0]

dp_dq = jac_explicit

dp_dq @ dq_dm

tensor([[-2.3906e-05,  1.5433e-05,  2.1243e-05, -3.0690e-05],
        [ 8.0282e-06, -5.1830e-06, -7.1341e-06,  1.0307e-05],
        [ 2.8990e-06, -1.8716e-06, -2.5761e-06,  3.7217e-06]],
       grad_fn=<MmBackward0>)

In [26]:
import time
n = 100

motor_control = torch.zeros(1,args.motor_dim).to(args.device)
start_time = time.time()


for _ in range(n):
    dp_dm, p = jacobian(partial(forward_model, model), motor_control)
    dp_dm = dp_dm[:,0,:]
    
    # print(dp_dm)
end_time = time.time()

autograd_time = end_time-start_time
print("[AutoGrad]:{:2f}".format(autograd_time))



start_time = time.time()
def forward_q(model:PRIMNET, motor_control):
    motor_control = model.normalize(motor_control)

    # Forward
    act_embeds = model.ACT_EMBED.layers(motor_control)
    q_values = model.FK_LAYER.forward_q(act_embeds)[0]
    return q_values

for _ in range(n):
    dq_dm,q_values_tuple = jacobian(partial(forward_q, model), motor_control)
    dq_dm = dq_dm[:,0,:].detach(); q_values = q_values_tuple[0].detach().unsqueeze(0)

    
    @ torch.jit.script_if_tracing
    def kinematic_grad(model:PRIMNET, q_values):
        with torch.no_grad():
            joint_se3 = model.FK_LAYER.forward_kinematics(q_values)

            joint_position =  model.t2p(joint_se3, OUTPUT_NORMALIZE=False)[0,:,:,0]
            joint_rotation = t2r(joint_se3[0])

            EE_pos = joint_position[-1]

            jac_explicit = torch.zeros(3, len(model.FK_LAYER.joints))

            for idx,joint in enumerate(model.FK_LAYER.joints):
                if isinstance(joint, Fjoint):
                    continue
                elif isinstance(joint, Tjoint):
                    continue
                elif isinstance(joint, Rjoint):
                    pos_diff = EE_pos - joint_position[idx]
                    jac_explicit[:, idx] = torch.cross(joint_rotation[idx] @ joint.axis.data[:,0], pos_diff)
                    # print('here')
                elif isinstance(joint, Pjoint):
                    pos_diff = EE_pos - joint_position[idx]
                    jac_explicit[:,idx] = joint_rotation[idx] @ joint.axis.data[:,0]
        
        return jac_explicit

    jac_explicit = kinematic_grad(model,q_values)
    dp_dm = (jac_explicit @ dq_dm).numpy()
    # print(dp_dm)

end_time = time.time()
explicit_time = end_time-start_time
print("[Explicit]:{:2f}".format(explicit_time))

print("Result: [Explicit] is faster than [AutoGrad] by {:2f} times".format(autograd_time/explicit_time))



[AutoGrad]:0.866574
[Explicit]:0.516586
Result: [Explicit] is faster than [AutoGrad] by 1.677501 times


In [28]:
dq_dm

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 8.5829e-06, -5.5411e-06, -7.6270e-06,  1.1019e-05],
        [ 3.4874e-05, -2.2515e-05, -3.0990e-05,  4.4772e-05],
        [-1.7198e-05,  1.1103e-05,  1.5283e-05, -2.2079e-05],
        [ 4.8425e-07, -3.1263e-07, -4.3031e-07,  6.2168e-07],
        [ 1.2597e-05, -8.1328e-06, -1.1194e-05,  1.6172e-05],
        [ 7.5818e-06, -4.8948e-06, -6.7374e-06,  9.7336e-06]])